Movielens 영화 SBR
=======

Movielens 1M Dataset을 기반으로, Session based Recommendation 시스템을 제작해 보겠습니다.

- Movielens 1M Dataset     
    https://www.kaggle.com/odedgolden/movielens-1m-dataset
- Session based Recommendation    
    세션 데이터를 기반으로 유저가 다음에 클릭 또는 구매할 아이템을 예측하는 추천을 말합니다.
    
-21.11.11-

In [1]:
# 데이터 확인
!ls -l ~/aiffel/Exploration/E12/data

total 4
lrwxrwxrwx 1 root root   31 Nov  4 01:35 dataset-README.txt -> /aiffel/data/dataset-README.txt
lrwxrwxrwx 1 root root   23 Nov  4 01:35 movies.dat -> /aiffel/data/movies.dat
drwxr-xr-x 2 root root 4096 Nov  4 03:19 processed
lrwxrwxrwx 1 root root   24 Nov  4 01:35 ratings.dat -> /aiffel/data/ratings.dat
lrwxrwxrwx 1 root root   19 Nov  4 01:35 README -> /aiffel/data/README
lrwxrwxrwx 1 root root   22 Nov  4 01:35 users.dat -> /aiffel/data/users.dat
lrwxrwxrwx 1 root root   31 Nov  4 01:35 yoochoose-buys.dat -> /aiffel/data/yoochoose-buys.dat
lrwxrwxrwx 1 root root   33 Nov  4 01:35 yoochoose-clicks.dat -> /aiffel/data/yoochoose-clicks.dat
lrwxrwxrwx 1 root root   31 Nov  4 01:35 yoochoose-test.dat -> /aiffel/data/yoochoose-test.dat


In [2]:
import datetime as dt
from pathlib import Path
import os

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 데이터 설명(README)를 읽어 봅니다. 
f = open(os.getenv('HOME')+'/aiffel/Exploration/E12/data/README', 'r')
while True:
    line = f.readline()
    if not line: break
    print(line)
f.close()

SUMMARY




These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 

made by 6,040 MovieLens users who joined MovieLens in 2000.



USAGE LICENSE




Neither the University of Minnesota nor any of the researchers

involved can guarantee the correctness of the data, its suitability

for any particular purpose, or the validity of results based on the

use of the data set.  The data set may be used for any research

purposes under the following conditions:



     * The user may not state or imply any endorsement from the

       University of Minnesota or the GroupLens Research Group.



     * The user must acknowledge the use of the data set in

       publications resulting from the use of the data set

       (see below for citation information).



     * The user may not redistribute the data without separate

       permission.



     * The user may not use this information for any commercial or

       revenue-bearing purposes without first obtaining permi

In [3]:
data_path = Path(os.getenv('HOME')+'/aiffel/Exploration/E12/data/') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


- 여기서 이전 실습내역과 가장 크게 다른 부분은 바로 SessionID 대신 UserID 항목이 들어갔다는 점입니다. 이 데이터셋은 명확한 1회 세션의 SessionID를 포함하지 않고 있습니다. 그래서 이번에는 UserID가 SessionID 역할을 해야 합니다.

- Rating 정보가 포함되어 있습니다. 이전 실습내역에서는 이런 항목이 포함되어 있지 않았으므로, 무시하고 제외할 수 있습니다. 하지만, 직전에 봤던 영화가 맘에 들었는지 여부가 비슷한 영화를 더 고르게 하는 것과 상관이 있을 수도 있습니다. 아울러, Rating이 낮은 데이터를 어떻게 처리할지도 고민해야 합니다.

- Time 항목에는 UTC time 가 포함되어, 1970년 1월 1일부터 경과된 초단위 시간이 기재되어 있습니다.

# 1. 데이터의 전처리

위와 같이 간단히 구성해 본 데이터셋을 꼼꼼이 살펴보면서 항목별 기본분석, session length, session time, cleaning 등의 작업을 진행합니다.    
특히, 이 데이터셋에서는 Session이 아닌 UserID 단위로 데이터가 생성되어 있으므로, 이를 Session 단위로 어떻게 해석할지에 주의합니다.